<h4>Librerías</h4>

In [ ]:
import os
import arcpy

<h4>Creación de base de datos y parametrización de rutas</h4>

In [ ]:
ruta_geojson = r'C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo'
arcpy.env.workspace = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb"

ruta_vulnerabilidad_urbana = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Vulnerabilidad_Urbana"
ruta_vulnerabilidad_rural = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Vulnerabilidad_Rural"

ruta_amenaza_urbano = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Amenaza_Urbano"
ruta_amenaza_rural = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Amenaza_Rural"

ruta_riesgo_urbano = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Riesgo_Urbano"
ruta_riesgo_rural = r"C:\Users\rodian.saby\OneDrive\Documentos\docsProyectos\3.PGAR\Insumos\Capas_Geograficas\_10_Analisis_Vulnerabilidad_Riesgo\_10_Analisis_Vulnerabilidad_Riesgo.gdb\Riesgo_Rural"


# Creación de Base de Datos
if arcpy.Exists(arcpy.env.workspace):
    for capas in arcpy.ListFeatureClasses():
        arcpy.management.Delete(capas)
        print("Se borra la información registrada en la capa: {0}".format(capas))
else:
    arcpy.management.CreateFileGDB(arcpy.env.workspace, '_10_Analisis_Vulnerabilidad_Riesgo')

<h4>Parametrización de las rutas donde se localizan los .geojson</h4>

In [ ]:
nombre_geojson = os.listdir(ruta_geojson)
lista_ruta_archivo = []
for rutas in nombre_geojson:
    lista_ruta_archivo.append(os.path.join(ruta_geojson, rutas))

<h4>Conversión de .geojson a Feature Class</h4>

In [ ]:
for generacion_feature in lista_ruta_archivo:
    nombre_geojson = os.path.split(generacion_feature)
    nombre_geojson_parametrizado = (nombre_geojson[1].split(".")[0]).replace(" ","_")
    if nombre_geojson_parametrizado != '_10_Analisis_Vulnerabilidad_Riesgo':
        salida_feature = os.path.join(arcpy.env.workspace, nombre_geojson_parametrizado)
        arcpy.conversion.JSONToFeatures(generacion_feature, salida_feature, 'POLYGON')
        print("geojson {0} convertido al feature {1}".format(nombre_geojson, nombre_geojson_parametrizado))
    else:
        print("Es la base de datos")

<h4>Creación y registro de nombre del feature class en cada una de las capas convertidas</h4>

In [ ]:
lista_capas_convertidas = arcpy.ListFeatureClasses()
nombre_campo = 'nombre_capa'

for features in lista_capas_convertidas:
    if features != 'Vulnerabilidad_Urbana' or features != 'Vulnerabilidad_Rural' or features != 'Amenaza_Urbana' or features != 'Amenaza_Rural' or features != 'Riesgo_Urbana' or features != 'Riesgo_Rural':
        descripcion = arcpy.Describe(features)
        expresion = "'" + descripcion.name +"'"
        arcpy.management.AddField(features, nombre_campo, 'TEXT', field_length = 100, field_alias = 'Nombre Capa Origen')
        arcpy.management.CalculateField(features, nombre_campo, expresion, expression_type = 'PYTHON3')
        print("Se actualiza la capa {0}".format(features))
    else:
        print("No actualizado")

<h4>Unificación de capa por temática</h4>

In [ ]:
capa_vulnerabilidad_urbana = 'Vulnerabilidad_Urbana'
capa_vulnerabilidad_rural = 'Vulnerabilidad_Rural'
capa_amenaza_urbano = 'Amenaza_Urbano'
capa_amenaza_rural = 'Amenaza_Rural'
capa_riesgo_urbano = 'Riesgo_Urbano'
capa_riesgo_rural = 'Riesgo_Rural'

lista_capas_vulnerabilidad_urbana = []
lista_capas_vulnerabilidad_rural = []
lista_capas_amenaza_urbano = []
lista_capas_amenaza_rural = []
lista_capas_riesgo_urbano = []
lista_capas_riesgo_rural = []

for features in lista_capas_convertidas:    
    if 'Vulnerabilidad_Urbano' in features:
        lista_capas_vulnerabilidad_urbana.append(features)
    elif 'Vulnerabilidad_Rural' in features:
        lista_capas_vulnerabilidad_rural.append(features)
    elif 'Amenaza_Urbano' in features:
        lista_capas_amenaza_urbano.append(features)
    elif 'Amenaza_Rural' in features:
        lista_capas_amenaza_rural.append(features)
    elif 'Riesgo_Urbano' in features:
        lista_capas_riesgo_urbano.append(features)
    elif 'Riesgo_Rural' in features:
        lista_capas_riesgo_rural.append(features)
    else:
        print("La capa {0}, no se unifico".format(features))
        
arcpy.management.Merge(lista_capas_vulnerabilidad_urbana, capa_vulnerabilidad_urbana)
print("Capa {0} unificada en {1}".format(features, capa_vulnerabilidad_urbana))

arcpy.management.Merge(lista_capas_vulnerabilidad_rural, capa_vulnerabilidad_rural)
print("Capa {0} unificada en {1}".format(features, capa_vulnerabilidad_rural))

arcpy.management.Merge(lista_capas_amenaza_urbano, capa_amenaza_urbano)
print("Capa {0} unificada en {1}".format(features, capa_amenaza_urbano))

arcpy.management.Merge(lista_capas_amenaza_rural, capa_amenaza_rural)
print("Capa {0} unificada en {1}".format(features, capa_amenaza_rural))

arcpy.management.Merge(lista_capas_riesgo_urbano, capa_riesgo_urbano)
print("Capa {0} unificada en {1}".format(features, capa_riesgo_urbano))

arcpy.management.Merge(lista_capas_riesgo_rural, capa_riesgo_rural)
print("Capa {0} unificada en {1}".format(features, capa_riesgo_rural))

